In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 

In [ ]:
df = pd.read_csv('data/xente/training.csv', parse_dates=['TransactionStartTime'], dtype={'ProductId': 'category','ProductCategory': 'category','ChannelId': 'category','PricingStrategy': 'category'}, index_col='TransactionId').drop(['CountryCode', 'CurrencyCode'], axis=1)

In [ ]:
df_var = pd.read_csv('data/xente/Xente_variable_Definitions.csv')

In [ ]:
df_var

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
#set TransactionId as index, as label 
#set as categorical ProductId, ProductCategory, ChannelId
#ChannelId could be interesting 
#Investigate Amount / Value --> why is Value different sometimes
#Investigate & transform TransactionStartTime
#Random Forrest?

In [ ]:
#convert datatypes if we deem necessary especially 'CountryCode' 

In [ ]:
#pairplot of our precleaned df

sns.pairplot(df)

In [ ]:
# our dataset is highly imbalanced --> todo: decide on how we want to address this 
# possibly use Amount as predictor for basemodel --> might ignore large amount of small fraudulent transactions though

# Our Metric
# 

In [ ]:
#example of Milestone 1, have them add this to milestone description:
#I think transactions that happen at midnight are likely to be fraudulent

In [ ]:

#Fraud Detection: (?)

#Value of Product: (?)

#Goal : Find fraudulent transactions, save money, avoid reputation damage and prevent money laundering.

#Prediction: 'Transaction is fraudulent'
#How to handle the imbalanced dataset: SMOTE

#Evaluation Metric:
# f1-score (recommended and given by Zindi)
# Matthew Coefficient Correlation Score
# AUC - Precision-Recall Curve

#Baseline Model: (tbd)

#Score:
# f1-score = 0.1
# Matthew Coefficient Correlation Score = (tbd)
# AUC - Precision-Recall Curve = (tbd)


# ridge, lasso & elasticnet 

In [ ]:
df['BatchId'].unique()

In [ ]:
df['AccountId'].unique()

In [ ]:
df['SubscriptionId'].unique()

In [ ]:
df['CurrencyCode'].unique()

In [ ]:
#only '256' --> Uganda --> no added Information --> drop this 
df['CountryCode'].unique()

In [ ]:
#keep this for now 
df['ProductCategory'].unique()

In [ ]:
#
df.groupby(['ProductCategory', 'FraudResult'])['FraudResult'].count()

In [ ]:
df.groupby('FraudResult')['FraudResult'].count()

In [ ]:
df.info()

In [ ]:
df_s = pd.read_csv('data/xente/sample_submission.csv')

In [ ]:
df_s